In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
_base_ = '../../data/2013_vast_challenge/mc3_netflow/nf/'
df_orig = pl.concat([pl.read_csv(_base_ + 'nf-chunk1.csv'), pl.read_csv(_base_ + 'nf-chunk2.csv'), pl.read_csv(_base_ + 'nf-chunk3.csv')])
# df_orig = pl.concat([pl.read_csv(_base_ + 'nf-chunk2.csv')])
df_orig = df_orig.rename({'TimeSeconds':'secs',                  'parsedDate':'timestamp',                'dateTimeStr':'timestamp_str',
                          'ipLayerProtocol':'pro_str',           'ipLayerProtocolCode':'pro',             'firstSeenSrcIp':'sip',
                          'firstSeenDestIp':'dip',               'firstSeenSrcPort':'spt',                'firstSeenDestPort':'dpt',
                          'moreFragments':'mfrag',               'contFragments':'cfrag',                 'durationSeconds':'dur',
                          'firstSeenSrcPayloadBytes':'soct_pay', 'firstSeenDestPayloadBytes':'doct_pay',  'firstSeenSrcTotalBytes':'soct',
                          'firstSeenDestTotalBytes':'doct',      'firstSeenSrcPacketCount':'spkt',        'firstSeenDestPacketCount':'dpkt',
                          'recordForceOut':'out'})
df_orig = rt.columnsAreTimestamps(df_orig, 'timestamp')

In [2]:
g             = rt.createNetworkXGraph(df_orig, [('sip','dip')])
_communities_ = nx.community.louvain_communities(g)

In [ ]:
_dfs_  = []
_svgs_ = []
for _community_ in _communities_:
    _df_ = df_orig.filter(pl.col('sip').is_in(_community_) & pl.col('dip').is_in(_community_))
    _cd_ = rt.chordDiagram(_df_, [('sip','dip')], link_style='bundled', color_by=rt.createTField('sip', 'ipv4_cidr_16'), link_color='vary', link_opacity=0.2, beta=0.75,
                           skeleton_algorithm='kmeans', track_routes=True, w=512, h=512, equal_size_nodes=True)
    _svgs_.append(_cd_), _dfs_.append(_df_)
rt.table(_svgs_, per_row=3)

In [ ]:
rt.chordDiagram(_dfs_[2], [('sip','dip')], link_style='bundled', skeleton_algorithm='kmeans', equal_size_nodes=True)

In [ ]:
_cd_ = rt.chordDiagram(df_orig, [('sip','dip')], 
                       link_style='bundled', color_by=rt.createTField('sip', 'ipv4_cidr_16'), link_color='vary', link_opacity=0.2, beta=0.85,
                       skeleton_algorithm='kmeans', track_routes=True, w=768, h=768, equal_size_nodes=True)
_cd_

In [ ]:
df_route_histogram_lu = {'segment':[], 'count':[], 'sip':[], 'dip':[]}
pos_routes            = {}
for _segment_ in _cd_.track_routes_segments:
    _count_ = _cd_.track_routes_segments[_segment_]
    df_route_histogram_lu['segment'].append(str(_segment_))
    df_route_histogram_lu['sip']    .append(str(_segment_[0]))
    pos_routes[str(_segment_[0])] = _segment_[0]
    pos_routes[str(_segment_[1])] = _segment_[1]
    df_route_histogram_lu['dip']    .append(str(_segment_[1]))
    df_route_histogram_lu['count']  .append(_count_)
df_route_histogram = pd.DataFrame(df_route_histogram_lu)
#rt.histogram(df_route_histogram, bin_by='segment', count_by='count', w=800, h=15000)
_link_ = rt.linkNode(df_route_histogram, [('sip','dip')], pos=pos_routes, link_arrow=False, link_shape='curve', count_by='count', 
                     link_size='vary', link_size_max=16.0, link_size_min=1.0, w=600, h=600)
rt.tile([_link_, _cd_.skeleton_svg])